In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/data.csv')
print(df.head())

         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId CurrencyCode  CountryCode    ProviderId     ProductId  \
0  CustomerId_4406          UGX          256  ProviderId_6  ProductId_10   
1  CustomerId_4406          UGX          256  ProviderId_4   ProductId_6   
2  CustomerId_4683          UGX          256  ProviderId_6   ProductId_1   
3   CustomerId_988          UGX          256  ProviderId_1  ProductId_21   
4   CustomerId_988          UGX          256  ProviderId_4   ProductId_6   

      ProductCategory    ChannelId   Amount  Value  TransactionStart

In [3]:
import numpy as np

# Assuming 'TransactionStartTime' is in datetime format and we already converted it previously
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Recency: Time since the last transaction per user
current_date = df['TransactionStartTime'].max()
df['Recency'] = df.groupby('AccountId')['TransactionStartTime'].transform(lambda x: (current_date - x.max()).days)

# Frequency: Count of transactions per user
df['Frequency'] = df.groupby('AccountId')['TransactionId'].transform('count')

# Monetary: Total transaction amount per user
df['Monetary'] = df.groupby('AccountId')['Amount'].transform('sum')

# Stability: Standard deviation of transaction amount per user
df['Stability'] = df.groupby('AccountId')['Amount'].transform('std')
df['Stability'].fillna(0, inplace=True)  # Replace NaN values for users with only one transaction


/tmp/ipykernel_19349/3946805259.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Stability'].fillna(0, inplace=True)  # Replace NaN values for users with only one transaction


In [4]:
# Calculate RFMS score
# Normalizing the RFMS components to bring them to a similar scale
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['Recency', 'Frequency', 'Monetary', 'Stability']] = scaler.fit_transform(df[['Recency', 'Frequency', 'Monetary', 'Stability']])

# Assign a simple weighted RFMS score (weights can be adjusted based on domain knowledge)
df['RFMS_Score'] = 0.25 * df['Recency'] + 0.25 * df['Frequency'] + 0.25 * df['Monetary'] + 0.25 * df['Stability']

# Classify users: Good (RFMS_Score >= 0.5) and Bad (RFMS_Score < 0.5)
df['User_Label'] = np.where(df['RFMS_Score'] >= 0.5, 'Good', 'Bad')


In [6]:
# Check the distribution of 'User_Label'
print(df['User_Label'].value_counts())


User_Label
Bad    95662
Name: count, dtype: int64


In [7]:
# Calculate the median RFMS score to use as the threshold for classification
threshold = df['RFMS_Score'].median()

# Classify users again based on the updated threshold
df['User_Label'] = np.where(df['RFMS_Score'] >= threshold, 'Good', 'Bad')

# Verify the distribution of labels again
print(df['User_Label'].value_counts())


User_Label
Good    47883
Bad     47779
Name: count, dtype: int64


In [11]:
import numpy as np

# Sample DataFrame creation (replace this with your actual data loading step)
# df = pd.read_csv('your_data.csv')

# Ensure necessary columns are numeric
woe_features = ['Recency', 'Frequency', 'Monetary', 'Stability']
target = 'User_Label'

for feature in woe_features:
    # Convert to numeric, errors='coerce' will turn non-convertible values to NaN
    df[feature] = pd.to_numeric(df[feature], errors='coerce')

# Convert target to binary if not already
df[target] = pd.to_numeric(df[target], errors='coerce')
df[target] = df[target].fillna(0).astype(int)  # fill NaN with 0 and ensure binary

def calculate_woe(df, feature, target):
    df_woe = df[[feature, target]].copy()
    df_woe['Total'] = 1

    # Group by feature value and calculate Good (0) and Bad (1) counts
    grouped = df_woe.groupby(feature).agg({target: ['sum', 'count']})
    grouped.columns = ['Bad', 'Total']
    grouped['Good'] = grouped['Total'] - grouped['Bad']

    # Calculate Distribution of Good and Bad
    grouped['Dist_Good'] = grouped['Good'] / grouped['Good'].sum()
    grouped['Dist_Bad'] = grouped['Bad'] / grouped['Bad'].sum()

    # Handle division by zero and infinite values
    grouped['WoE'] = np.where(grouped['Dist_Bad'] == 0, np.inf, np.log(grouped['Dist_Good'] / grouped['Dist_Bad']))

    return grouped[['WoE']].reset_index()

# Fit the WoE model on features and add to the DataFrame
for feature in woe_features:
    # Calculate WoE for each feature
    woe_values = calculate_woe(df, feature, target)
    
    # Merge WoE values back to the original DataFrame
    df = df.merge(woe_values, on=feature, how='left', suffixes=('', f'_{feature}_woe'))

# Check the resulting DataFrame
print(df.head())


         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId CurrencyCode  CountryCode    ProviderId     ProductId  \
0  CustomerId_4406          UGX          256  ProviderId_6  ProductId_10   
1  CustomerId_4406          UGX          256  ProviderId_4   ProductId_6   
2  CustomerId_4683          UGX          256  ProviderId_6   ProductId_1   
3   CustomerId_988          UGX          256  ProviderId_1  ProductId_21   
4   CustomerId_988          UGX          256  ProviderId_4   ProductId_6   

      ProductCategory  ...   Recency  Frequency  Monetary Stability 

In [13]:
# Export the processed DataFrame to a CSV file
df.to_csv('processed_data.csv', index=False)
print("Processed data exported as 'processed_data.csv'.")


Processed data exported as 'processed_data.csv'.


/tmp/ipykernel_19349/3097922745.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
